In [69]:
#install required modules and libraries
#! pip install pyarrow

import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

#iterate over csv files into pandas selecting only the required columns
#create a compressed parquet dataset by year>month>region for a smaller footprint and faster querying

def weatherResults():
    for file in os.listdir():
        if file.endswith(".csv"):
            file1 = file
            df = pd.read_csv(os.path.abspath(file1))


            df = df[['ForecastSiteCode','ObservationTime','ObservationDate','ScreenTemperature','SiteName','Region']]
            df = df.sort_values(by=['ForecastSiteCode','ObservationDate','ObservationTime'])
            df = df.reset_index(drop=True)
            df['ObsYear'] = pd.DatetimeIndex(df['ObservationDate']).year
            df['ObsMonth'] = pd.DatetimeIndex(df['ObservationDate']).month
            df['ObsDay'] = pd.DatetimeIndex(df['ObservationDate']).day

            table = pa.Table.from_pandas(df)

            #create files for testing 

            #file1 = file1.replace(".csv",".")
            #file2 = file1 + 'parquet.snappy'
            #pq.write_table(table, file2,compression='snappy')

            return pq.write_to_dataset(table,root_path='weather_results',partition_cols=['ObsYear','ObsMonth','Region'])
        

def maxTemp():
    #read parquet dataset into pandas dataframe and filter for max temp - print back required columns

    weather_data = pq.ParquetDataset('weather_results/')
    table = weather_data.read()
    weather_table_df = table.to_pandas()

    weather_result = weather_table_df.loc[weather_table_df['ScreenTemperature'].idxmax()]

    temp = weather_result['ScreenTemperature']
    date = weather_result['ObservationDate'][:9]
    region = weather_result['Region']

    print("Which date was the hottest day? ",date)
    print("") 
    print("What was the temperature on that day? ",temp)
    print("")        
    print("In which region was the hottest day? ",region) 


In [37]:
#Testing


#open files in excel note down row details for maximum temps
df1 = pd.read_csv(os.path.abspath('weather.20160201.csv'))
df2 = pd.read_csv(os.path.abspath('weather.20160301.csv'))

print(df1['ScreenTemperature'].max(),df2['ScreenTemperature'].max())

15.6 15.8


In [38]:
#check schema of dataset as table
table.schema

ForecastSiteCode: int64
  -- field metadata --
  PARQUET:field_id: '1'
ObservationTime: int64
  -- field metadata --
  PARQUET:field_id: '2'
ObservationDate: string
  -- field metadata --
  PARQUET:field_id: '3'
ScreenTemperature: double
  -- field metadata --
  PARQUET:field_id: '4'
SiteName: string
  -- field metadata --
  PARQUET:field_id: '5'
ObsDay: int64
  -- field metadata --
  PARQUET:field_id: '6'
ObsYear: dictionary<values=int64, indices=int32, ordered=0>
ObsMonth: dictionary<values=int64, indices=int32, ordered=0>
Region: dictionary<values=string, indices=int32, ordered=0>
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 822

In [21]:
#check parquet file metadata and schema

wather_20160301_file = pq.ParquetFile(os.path.abspath('weather.20160301.parquet.snappy'))
wather_20160201_file = pq.ParquetFile(os.path.abspath('weather.20160201.parquet.snappy'))

In [19]:
wather_20160301_file.metadata

  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 9
  num_rows: 101442
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 5435

In [22]:
wather_20160201_file.metadata

  created_by: parquet-cpp version 1.5.1-SNAPSHOT
  num_columns: 9
  num_rows: 93255
  num_row_groups: 1
  format_version: 1.0
  serialized_size: 5435

In [23]:
wather_20160301_file.schema

required group field_id=0 schema {
  optional int64 field_id=1 ForecastSiteCode;
  optional int64 field_id=2 ObservationTime;
  optional binary field_id=3 ObservationDate (String);
  optional double field_id=4 ScreenTemperature;
  optional binary field_id=5 SiteName (String);
  optional binary field_id=6 Region (String);
  optional int64 field_id=7 ObsYear;
  optional int64 field_id=8 ObsMonth;
  optional int64 field_id=9 ObsDay;
}

In [24]:
wather_20160201_file.schema

required group field_id=0 schema {
  optional int64 field_id=1 ForecastSiteCode;
  optional int64 field_id=2 ObservationTime;
  optional binary field_id=3 ObservationDate (String);
  optional double field_id=4 ScreenTemperature;
  optional binary field_id=5 SiteName (String);
  optional binary field_id=6 Region (String);
  optional int64 field_id=7 ObsYear;
  optional int64 field_id=8 ObsMonth;
  optional int64 field_id=9 ObsDay;
}

In [65]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import os

In [68]:
def weather(directorypath):
    return os.chdir(directorypath)

In [66]:
def pqdataSet():
    for file in os.listdir():
        if file.endswith(".csv"):
            file1 = file
            df = pd.read_csv(os.path.abspath(file1))

In [67]:
pqdataSet()

In [43]:
directorypath = "\\Users\walea\Downloads\Data_Engineer_Test_Green_Flag"
#directorypath = directorypath.replace("\\","//")
#directorypath 

In [38]:
weather('\Users\walea\Downloads\Data_Engineer_Test_Green_Flag')

raw_s = r'{}'.format('\Users\walea\Downloads\Data_Engineer_Test_Green_Flag')

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-1: truncated \UXXXXXXXX escape (<ipython-input-38-c22d8caf8d74>, line 1)

In [54]:
os.getcwd()

'C:\\Users\\walea\\Downloads\\Data_Engineer_Test_Green_Flag'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (<ipython-input-53-096485d4ef89>, line 1)

In [72]:
import pandas as pd
import glob, os


class Weather:

    def __init__(self, source_path, destination_path):
        self.source_path = source_path
        self.destination_path = destination_path
        self.weather_df = None
        self.combined_weather_df = None
        self.generate_weather_df()

    def generate_weather_df(self):
        self.read_weather_csv_data_files()

    def read_weather_csv_data_files(self):
        '''
        Read required data columns from csv files source location and concatenate them
        in to a panda dataframe.
        '''
        files = glob.glob(os.path.join(self.source_path, "*.csv"))
        weather_data_csv = [pd.read_csv(f, usecols=["ObservationDate", "ScreenTemperature", "Region"]
                                        , converters={'ScreenTemperature': lambda x: (x.replace('-99', '0')),
                                                      'ObservationDate': lambda x: (pd.Timestamp(x).date())}
                                        ) for f in files]
        weather_data_csv_files = pd.concat(weather_data_csv, ignore_index=True)
        weather_data_csv_files['ScreenTemperature'] = pd.to_numeric(weather_data_csv_files['ScreenTemperature'])
        weather_data_csv_files.style.format({'ScreenTemperature': '{:,.1f}'.format})
        self.save_df_to_parquet_file(weather_data_csv_files)
        return weather_data_csv_files

    def save_df_to_parquet_file(self, weather_data_csv):
        '''
        Save panda dataframe to parquet file to output path
        :param weather_data_csv.
        '''
        output_path = os.path.join(self.destination_path, 'weather_data.parquet')
        weather_data_csv.to_parquet(output_path)
        self.load_parquet_file_to_df()

    def load_parquet_file_to_df(self):
        '''
        Load data to panda dataframe from parquet file
        '''
        self.weather_df = pd.read_parquet(self.destination_path)

    def hottest_day_date(self):
        '''
        Calculate and hottest day date
        :return: series: pandas series returning hottest day date
        '''
        grouped_by_date = self.weather_df.groupby(['ObservationDate'], as_index=False)
        average_temperature_for_dates = grouped_by_date.mean()
        hottest_day_date = average_temperature_for_dates[['ObservationDate']].loc[
            average_temperature_for_dates['ScreenTemperature'].idxmax()]
        return hottest_day_date

    def hottest_day_average_temperature(self):
        '''
        Calculate and hottest day temperature
        :return: series: pandas series returning hottest day temperature
        '''
        grouped_by_date = self.weather_df.groupby(['ObservationDate'], as_index=False)
        average_temperature_for_dates = grouped_by_date.mean()
        hottest_day_temperature = average_temperature_for_dates[['ScreenTemperature']].loc[
            average_temperature_for_dates['ScreenTemperature'].idxmax()]
        return hottest_day_temperature

    def hottest_day_average_temperature_by_region(self):
        '''
        Calculate and hottest day temperature by region
        :return: series: pandas series returning hottest day by region
        '''
        grouped_by_date_region = self.weather_df.groupby(['ObservationDate', 'Region'], as_index=False)
        average_temperature_for_dates_region = grouped_by_date_region.mean()
        hottest_day_temperature_region = average_temperature_for_dates_region[['ObservationDate', 'Region']].loc[
            average_temperature_for_dates_region['ScreenTemperature'].idxmax()]
        return hottest_day_temperature_region

    def get_results(self):
        '''
        Concatinate all series data to a Pandas dataframe
        :return: dataframe: pandas dataframe returning hottest day date
                            hottest day temperature and hottest day by region
        '''
        pd.set_option('display.max_columns', 3)
        self.combined_weather_df = pd.concat([self.hottest_day_date(), self.hottest_day_average_temperature(),
                                              self.hottest_day_average_temperature_by_region()], sort=True, axis=1,
                                             ignore_index=True)

        self.combined_weather_df.rename(columns={self.combined_weather_df.columns[0]: "hottest_day_date",
                                                 self.combined_weather_df.columns[1]: "hottest_day_temperature",
                                                 self.combined_weather_df.columns[2]: "hottest_day_by_region"}
                                        , inplace=True)
        return self.combined_weather_df

In [78]:
new_weather = Weather('/Users/walea/Desktop/New1','/Users/walea/Desktop/other')

In [86]:
new_weather.get_results()

,hottest_day_date,hottest_day_temperature,hottest_day_by_region
ObservationDate,2016-02-01,NaN,2016-02-21
Region,NaN,NaN,East of England
ScreenTemperature,NaN,9.153981,NaN
